In [26]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [27]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [29]:
class AIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = ai_message.additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, tool_call: dict) -> ChatMessage:
        id_ = tool_call.id
        function_call = tool_call.function
        tool = self._tools[function_call.name]
        output = tool(**json.loads(function_call.arguments))
        print(f"> Calling tool: {function_call.name}")
        return ChatMessage(
            name=function_call.name,
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call.name,
            },
        )

In [30]:
agent = AIAgent(tools=[multiply_tool, add_tool])
agent.chat("What is 2123 * 215123")

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/etc/ssl/certs/ca-certificates.crt'
load_verify_locations cafile='/etc/ssl/certs/ca-certificates.crt'
load_verify_locations cafile='/etc/ssl/certs/ca-certificates.crt'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is 2123 * 215123'}], 'model': 'gpt-3.5-turbo-0613', 'stream': False, 'temperature': 0.0, 'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(a: int, b: int) -> int\nMultiple two integers and returns the result integer', 'parameters': {'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}}}, {

'The product of 2123 multiplied by 215123 is 456,706,129.'

### OPENAIAgent

In [32]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [33]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/etc/ssl/certs/ca-certificates.crt'
load_verify_locations cafile='/etc/ssl/certs/ca-certificates.crt'
load_verify_locations cafile='/etc/ssl/certs/ca-certificates.crt'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is (121 * 3) + 42?'}], 'model': 'gpt-3.5-turbo-0613', 'stream': False, 'temperature': 0.1, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(a: int, b: int) -> int\nMultiple two integers and returns the result integer', 'parameters': {'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'i

### Agent with personality

In [34]:
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [35]:
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

response = agent.chat("Tell me a story")
print(response)

Added user message to memory: Tell me a story
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a Shakespearean writing assistant who speaks in a Shakespearean style. You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.\nHere are some example of Shakespeare\'s style:\n - Romeo, Romeo! Wherefore art thou Romeo?\n - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.\n - Shall I compare thee to a summer\'s day? Thou art more lovely and more temperate.\n'}, {'role': 'user', 'content': 'Tell me a story'}], 'model': 'gpt-3.5-turbo-0613', 'stream': False, 'temperature': 0.1, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(a: int, b: int) -> int\nMult